# Fake News Detector - Spanish Edition

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#data-cleaning" data-toc-modified-id="data-cleaning-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>data cleaning</a></span></li></ul></div>

In [1]:
# pandas & numpy:
import numpy as np
import pandas as pd

# visualization:
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
import plotly.express as px

# tokenization:
import re
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.porter import PorterStemmer
import string

# gensim:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora
import pyLDAvis.gensim_models

# scikit-learn:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV



C:\Users\Merche\anaconda3\envs\Da_Env1\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload


In [2]:
true_df=pd.read_csv("Data/onlytrue1000.csv")
true_df

,text
0,Algunas de las voces extremistas más conocida...
1,Después de casi dos años y medio de luchas po...
2,Dos periodistas birmanos de la agencia Reuter...
3,El Cuerpo Nacional de Policía ha detenido a c...
4,El desfile de la firma en Roma se convierte e...
...,...
995,El Consejo de Gobierno ha dado su visto bueno...
996,Investigadores valencianos han desarrollado u...
997,Los arrestados actuaban en coches y en establ...
998,El Rey ha encargado este miércoles a Pedro Sá...


In [3]:
fake_df=pd.read_csv("Data/onlyfakes1000.csv")
fake_df

,text
0,El suceso ha tenido lugar en Brasil. Un adole...
1,Estamos en la semana decisiva. Los expertos a...
2,Estudios científicos hay muchos. Unos nos int...
3,Ha sucedido en la ciudad de San José de Río P...
4,La fiesta en Sevilla por el vuelco electoral ...
...,...
995,Reenviado... Me acaban de llegar a casa por co...
996,"JULIO ANGUITA: ""El que debería estar en la cár..."
997,Otegi reclama la salida de prisión de todos lo...
998,Otegi reclama la salida de prisión de todos lo...


In [4]:
true_df["is_fake"]=0
fake_df["is_fake"]=1

In [5]:
df=pd.concat([true_df,fake_df],axis=0).reset_index(drop=True)
df


,text,is_fake
0,Algunas de las voces extremistas más conocida...,0
1,Después de casi dos años y medio de luchas po...,0
2,Dos periodistas birmanos de la agencia Reuter...,0
3,El Cuerpo Nacional de Policía ha detenido a c...,0
4,El desfile de la firma en Roma se convierte e...,0
...,...,...
1995,Reenviado... Me acaban de llegar a casa por co...,1
1996,"JULIO ANGUITA: ""El que debería estar en la cár...",1
1997,Otegi reclama la salida de prisión de todos lo...,1
1998,Otegi reclama la salida de prisión de todos lo...,1


In [6]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   text     2000 non-null   object
 1   is_fake  2000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 31.4+ KB


## data cleaning 

In [7]:
# Getting stopwords for spanish

stop_words = set(stopwords.words('spanish'))

In [8]:
# Define punctuations to exclude and lemmatizer:

exclude = set(string.punctuation)

In [9]:
# Creating text data cleaning function:

def clean(text,stop_words):
    stop_words = set(stopwords.words('spanish'))
    #stop_words.update(('from','to','cc','http', 're', 'www', 'com','subject','sent','email','u','ok','thanks','please','ect','dt','pm'))
    exclude = set(string.punctuation) 
    lemma = WordNetLemmatizer()
    porter= PorterStemmer()
    
    text=text.rstrip()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    stop_free = " ".join([i for i in text.lower().split() if((i not in stop_words) and (not i.isdigit()) and len(i)>3)])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    stem = " ".join(porter.stem(token) for token in normalized.split())
    
    return normalized

In [10]:
# Clean the emails in df

text_clean=[]
for text in df['text']:
    text_clean.append(clean(text, stop_words).split()) 
print(text_clean)

[['vox', 'extremistas', 'conocidas', 'cuentas', 'asociadas', 'movimiento', 'conocida', 'right', 'ligadas', 'grupos', 'antisemitas', 'fobos', 'sido', 'eliminadas', 'plumazo', 'redes', 'sociales', 'facebook', 'instagram', 'trat'], ['despu', 'casi', 'medio', 'luchas', 'ticas', 'judiciales', 'donald', 'trump', 'presentado', 'proyecto', 'inmigraci', 'unidos', 'proyecto', 'elaborado', 'yerno', 'asesor', 'donald', 'trump', 'jared', 'kushner', 'concreta', 'cosa'], ['periodistas', 'birmanos', 'agencia', 'reuters', 'liberados', 'martes', 'recibir', 'perd', 'presidente', 'tras', 'pasar', 'medio', 'prisi', 'revelar', 'matanza', 'rcito', 'minor', 'musulmana', 'rohingya', 'lone', 'kyaw'], ['cuerpo', 'nacional', 'polic', 'detenido', 'cuatro', 'persona', 'acusadas', 'formar', 'parte', 'grupo', 'criminal', 'dedicado', 'fico', 'seres', 'humanos', 'frontera', 'francoespa', 'gipuzkoa', 'lula', 'captaba', 'extranjeros', 'llegados', 'pateras'], ['desfile', 'firma', 'rom', 'convierte', 'libertad', 'mujeres',

In [12]:
# Apply the function to the dataframe:

df['clean_text'] = text_clean

In [13]:
# Now we can compare the original column "text" vs the column "clean_text" using the row 0 from df
# text:

df['text'][0]

' Algunas de las voces extremistas más conocidas de EE.UU., cuentas asociadas con al movimiento de la conocida como alt-right o ligadas a grupos antisemitas o xenófobos , han sido eliminadas de un plumazo de la redes sociales Facebook e Instagram. Se trat'

In [14]:
# clean_text:

print(df['clean_text'][0])

['vox', 'extremistas', 'conocidas', 'cuentas', 'asociadas', 'movimiento', 'conocida', 'right', 'ligadas', 'grupos', 'antisemitas', 'fobos', 'sido', 'eliminadas', 'plumazo', 'redes', 'sociales', 'facebook', 'instagram', 'trat']


In [16]:
# Creating a new column named clean_joined where we join the words into a string.

df['clean_joined'] = df['clean_text'].apply(lambda x: " ".join(x))

In [17]:
# Let's see how the text looks like after the cleaning process.

df['clean_joined'][0]

'vox extremistas conocidas cuentas asociadas movimiento conocida right ligadas grupos antisemitas fobos sido eliminadas plumazo redes sociales facebook instagram trat'